# Projeto - Lógica de Programação II

__Aluno:__ Gabriel Resende Miranda

## Informações iniciais

Importando bibliotecas necessárias:

In [1]:
from collections import namedtuple as nt
from functools import reduce
import csv

# Precisei importar essa biblioteca para copiar dois dicionários com método deepcopy.
from copy import deepcopy

# Precisei dessa biblioteca para verificar se meus aquivos externos existem.
from os.path import exists

Como o projeto consiste em um sistema de gerenciamento de vendas, não irei incluir funções que tratem do cadastro de produtos nem de controle de estoque. Para isso, __irei assumir que a minha loja em questão trata-se de uma loja de roupas__ e que ela vende os seguintes produtos:

In [2]:
# produtos = {"nome do produto1": preço1, "nome do produto2": preço2, ...}

catalogo_produtos = {
    "Camisa": 90,
    "Calça": 120,
    "Vestido": 200,
    "Saia": 70,
    "Sapato": 190,
    "Bolsa": 300,
    "Jaqueta": 350,
    "Chapéu": 40,
    "Óculos": 110,
    "Colar": 160,
}

Ao cadastrar uma venda, o vendedor deverá simplesmente passar o nome dos produtos em questão. O preço será buscado direto no dicionário acima.

Como o sistema será acessado apenas por funcionários, irei criar uma lista de tuplas com as informações dos 5 vendedores cadastrados na minha loja. Eles irão acessar meu sistema através de um login. As vendas registradas por eles automaticamente terão seus respectivos nomes.

In [3]:
vendedor = nt("Vendedor", ["nome", "senha"])

vendedor1 = vendedor(nome="Gabriel", senha="abc")
vendedor2 = vendedor(nome="João", senha="123")
vendedor3 = vendedor(nome="Maria", senha="456")
vendedor4 = vendedor(nome="Fernando", senha="efg")
vendedor5 = vendedor(nome="Daniela", senha="hjk")

# Criando uma lista de vendedores cadastrados:
vendedores = [vendedor1, vendedor2, vendedor3, vendedor4, vendedor5]

As informações sobre as vendas serão salvas no dicionário abaixo, que servirá como nosso banco de dados. Ele seguirá a seguinte estrutura:

```python
db_vendas = {"ID": {informações da venda}}
```

Um exemplo de registro se dará da seguinte forma:

```python
db_vendas[0] = {
    "produtos": ["Camisa", "Calça"],
    "valores": [90, 120],
    "quantidades": [1, 2],
    "cpf_comprador": ["12345678900"],
    "nome_vendedor": "Gabriel",
    "dia": 11,
    "mes": 6,
    "ano": 2023,
    "info_adicional": "Compra a prazo"
}
```

Ao iniciarmos o programa, `db_vendas` receberá todas as vendas cadastradas no aquivo `db_vendas.csv`.

Esse dicionário será uma espécie de <i>variável global</i>. Todas as operações das funções irão operar diretamente nele.

In [4]:
db_vendas = {}

## Funções

In [5]:
# Funções extras:

def importar_db() -> dict:
    """
        Importa o banco de dados salvo no arquivo "db_vendas.csv".
        * Retorna uma lista com os seguintes elementos:
            * Dicionário no estilo {ID: {informações da venda}}.
            * Último id identificado.
            * [{},0] caso não exista o arquivo "db_vendas.csv" ou ocorra algum erro.
    """
    try:
        if exists("db_vendas.csv"):
            with open("db_vendas.csv", 'r') as db:
                planilha = csv.reader(db, delimiter=',', lineterminator='\n')
                tabela = list(planilha)

            # Pegando o header.
            chaves = tabela[0]

            # Eliminando o header.
            tabela = tabela[1:]

            ids = [int(item[0]) for item in tabela]

            # Transformando cada linha em um dicionário com a nossa configuração padrão.
            # [{venda0}, {venda1}, ...]
            dici_itens = []
            for item in tabela:
                dici_itens.append(dict(zip(chaves, item)))

            for i in range(len(dici_itens)):
                # Removendo o id dos dicionários correspondentes às vendas individuais
                # porque eles serão as chaves do nosso dicionário db_vendas.
                del dici_itens[i]["id"]

                if dici_itens[i]["info_adicional"] != "Venda cancelada":
                    # Ajustando a lista de produtos.
                    dici_itens[i]["produtos"] = dici_itens[i]["produtos"].split("; ")

                    # Ajustando a lista de valores.
                    dici_itens[i]["valores"] = [
                        int(v) for v in dici_itens[i]["valores"].split("; ")
                    ]

                    # Ajustando a lista de quantidades.
                    dici_itens[i]["quantidades"] = [
                        int(v) for v in dici_itens[i]["quantidades"].split("; ")
                    ]

                    # Ajustando a data
                    dici_itens[i]["dia"] = int(dici_itens[i]["dia"])
                    dici_itens[i]["mes"] = int(dici_itens[i]["mes"])
                    dici_itens[i]["ano"] = int(dici_itens[i]["ano"])

            print("Arquivo encontrado!")
            return [dict(zip(ids, dici_itens)), ids[-1]]
        else:
            print("Arquivo \"db_vendas.csv\" não encontrado. Iniciaremos com db_vendas = {} e ultimo_id = 0.")
            return [{},0]
    except:
        print("Erro ao importar o bando de dados. Iniciaremos com db_vendas = {} e ultimo_id = 0.")
        return [{},0]

    
def login(usuario: str, senha: str) -> bool:
    """
        Função responsável pelo login do vendedor.
        * Recebe o nome de usuário e a senha.
        * Retorna se os dados batem com os vendores cadastrados na lista "vendedores".
    """
    usuario_senha = False
    for vendedor in vendedores:
        if vendedor.nome == usuario:
            if vendedor.senha == senha:
                usuario_senha = True
    return usuario_senha


def resposta_menu(*msg: str) -> int:
    """
        Verifica a opção do menu digitada pelo usuário.
        Caso o usuário não digite um valor válido, a função é chamada novamente de forma recursiva.
        * Recebe como parâmetro opcional uma mensagem de alerta.
    """
    if msg:
        print(msg[0])
        
    try:
        resposta = int(input("O que deseja realizar? Número correspondente às opções do menu:"))
        if resposta in range(0,12):
            return resposta
        else:
            return resposta_menu("Opção inválida.\n")
    except:
        return resposta_menu("Opção inválida.\n")

    
def verifica_produtos(input_produtos: str) -> list:
    """
        Verifica o input de produtos digitado pelo usuário.
        * Recebe o input.
        * Retorna uma lista:
            [False, "Input inválido.\\n", None], se os produtos digitados não foram encontrados no catalogo_produtos.
            [True, None, lista_produtos], se foram encontrados.
        Atenção! A primeira letra do nome de cada produto deve ser maiúscula!
    """
    try:
        lista_produtos = [p.replace(" ","") for p in input_produtos.split(",")]
        verificando_no_catalogo = lambda p: catalogo_produtos.get(p) != None
        tem_na_loja = list(map(verificando_no_catalogo, lista_produtos))
        if False in tem_na_loja:
            return [False, "Input inválido.\n", None]
        else:
            # Verificando se o usuário digitou um produto válido mais de uma vez.
            verificando_qtd_vezes = lambda p: lista_produtos.count(p) == 1
            tem_apenas_uma_vez = list(map(verificando_qtd_vezes, lista_produtos))
            
            if False in tem_apenas_uma_vez:
                return [False, "Digite cada produto apenas uma vez.\n", None]
            else:
                return [True, None, lista_produtos]
    except:
        return [False, "Input inválido.\n", None]

    
def verifica_qtd(input_qtd: str, qtd_produtos: int) -> list:
    """
        Verifica o input de quantidade de produtos digitado pelo usuário.
        * Recebe o input e a quantidade de produtos vendidos.
        * Retorna uma lista:
            [False, "Input inválido.\\n", None], se as quantidades digitadas não correspondem às quantidades de produtos vendidos.
            [True, None, lista_qtd], caso contrário.
    """
    try:
        lista_qtd = [int(q.replace(" ","")) for q in input_qtd.split(",")]
        if len(lista_qtd) == qtd_produtos:
            return [True, None, lista_qtd]
        else:
            return [False, "Input inválido.\n", None]
    except:
        return [False, "Input inválido.\n", None]

    
def verifica_data(input_data: str, data_completa = True) -> list:
    """
        Verifica o input de data digitado pelo usuário.
        * Recebe a data (que deve estar no formato dd/mm/aaaa por padrão).
            Caso data_completa == False, então input_data deve estar no formato mm/aaaa.
        * Retorna uma lista:
            [False, "Input inválido.\\n", None], se a data digitada não for válida.
            [True, None, lista_data], caso contrário.
    """
    try:
        lista_data = input_data.split("/")
        lista_data = [int(n) for n in lista_data]
        
        verificando_data = lambda num, range_data: num in range_data
        
        if data_completa:
            dia = verificando_data(lista_data[0], range(1,32))
            mes = verificando_data(lista_data[1], range(1,13))
            ano = verificando_data(lista_data[2], range(1900,2100))
        else:
            dia = True
            mes = verificando_data(lista_data[0], range(1,13))
            ano = verificando_data(lista_data[1], range(1900,2100))
        
        if False in [dia, mes, ano]:
            return [False, "Input inválido.\n", None]
        else:
            return [True, None, lista_data]
    except:
        return [False, "Input inválido.\n", None]


def verifica_id(input_id: str) -> list:
    """
        Verifica o input de id digitado pelo usuário.
        * Recebe o id.
        * Retorna uma lista:
            [False, "Input inválido.\\n", None], se o id digitado não for válido.
            [True, None, id], caso contrário.
    """
    try:
        if int(input_id) in db_vendas.keys():
            return [True, None, int(input_id)]
        else:
            return [False, "Input inválido.\n", None]
    except:
        return [False, "Input inválido.\n", None]


def verifica_info_adicional(input_info: str) -> list:
    """
        Verifica o input de "info_adicional" a ser cadastrada no banco de dados.
        * Recebe o input do usuário.
        * Retorna:
            [True, None, float(input_cotacao)], se input_info for diferente de "Venda cancelada" (valor reservado).
            [False, "Input inválido.\n", None], caso contrário.
    """
    try:
        if input_info != "Venda cancelada":
            return [True, None]
        else:
            return [False, "Input inválido.\n"]
    except:
        return [False, "Input inválido.\n"]

    
def verifica_cotacao_dolar(input_cotacao: str) -> list:
    """
        Verifica o input de cotação do dólar.
        * Recebe o input do usuário.
        * Retorna:
            [True, None], se input_info for um valor válido
            [False, "Input inválido.\n"], caso contrário.
    """
    try:
        return [True, None, float(input_cotacao)]
    except:
        return [False, "Input inválido.\n", None]    
    

def filtrar_vendas_ano_mes(ano: int, mes: int) -> list:
    """
        Filtra as vendas cadastradas no banco de dados db_vendas por ano e por mes.
        * Recebe ano e mês de interesse.
        * Retorna uma lista com as vendas filtradas.
    """
    try:
        if len(db_vendas) != 0:
            filtro_ano = lambda venda: venda["ano"] == ano
            # Obtendo uma lista de vendas separadas por ano.
            vendas_por_ano = list(filter(filtro_ano,
                                         [db_vendas[id_venda] for id_venda in db_vendas.keys()]
                                        )
                                 )
            filtro_mes = lambda venda: venda["mes"] == mes
            vendas_por_mes = list(filter(filtro_mes, vendas_por_ano))
            return vendas_por_mes
        else:
            return []
    except:
        print("Erro ao filtrar vendas por data (ano/mes).")
        return []


def filtrar_vendedor(dict_retorno: dict, venda_individual: dict) -> dict:
    """
        Filtra as vendas por vendedor.
        * Recebe um dicionário vazio (dict_retorno) e uma venda de interesse (venda_individual).
        * Retorna um dicionário com a seguinte estrutura:
            {"nome_do_vendedor": [{venda_realizada_por_ele}]}
    """
    try:
        nome_do_vendedor = venda_individual["nome_vendedor"]
        if nome_do_vendedor in dict_retorno.keys():
            dict_retorno[nome_do_vendedor].append(venda_individual)
        else:
            dict_retorno[nome_do_vendedor] = [venda_individual]
        return dict_retorno
    except:
        print("Erro ao filtrar vendas por vendedor.")
        return {} 

    
# Funções do menu:

# [0]
def exibe_menu() -> None:
    """
        Exibe o menu com a opções disponíveis para o usuário.
    """
    print("##### Menu #####\n")
    print("0. Exibe o menu novamente")
    print("1. Exibe o catálogo de produtos da loja")
    print("2. Cadastrar venda")
    print("3. Excluir venda")
    print("4. Exibir vendas cadastradas")
    print("5. Editar venda cadastrada")
    print("6. Calcular o valor total e o valor médio das vendas em um determinado mês")
    print("7. Vendedor do mês")
    print("8. Produto do mês")
    print("9. Salvar as vendas no arquivo db_vendas.csv")
    print("10. Converter os valores em dólares e salvar dados em um novo arquivo")
    print("\t* Essa operação salva também as novas vendas cadastradas no arquivo db_vendas.csv.")
    print("11. Sair\n")
    print("Não se esqueça de salvar as alterações (opção 9) antes de sair!\n")
    
    
# [1]
def exibe_produtos() -> None:
    """
        Exibe o catálogo de produtos cadastrados na loja.
    """
    for produto, valor in zip(catalogo_produtos.keys(), catalogo_produtos.values()):
        print(f". {produto}: R$ {valor},00")

        
# [2]
def cadastrar_venda(id_venda: int,
                    produtos: list,
                    quantidades: list,
                    nome_vendedor: str,
                    cpf: str,
                    data: str,
                    info: str) -> bool:
    """
        Cadastra a venda solicitada pelo usuário.
        * Recebe informações da venda.
        * Cria um dicionário com as seguintes chaves:
            "produtos",
            "valores",
            "quantidades",
            "cpf_comprador",
            "nome_vendedor",
            "dia",
            "mes",
            "ano",
            "info_adicional"
        * Armazena o dicionário criado no banco de dados "db_vendas".
        * As chaves do dicionário "db_vendas" são ID de cada venda.
    """
    try:        
        precos = [catalogo_produtos[p] for p in produtos]
        
        db_vendas[id_venda] = {
            "produtos": produtos,
            "valores": precos,
            "quantidades": quantidades,
            "cpf_comprador": cpf,
            "nome_vendedor": nome_vendedor,
            "dia": data[0],
            "mes": data[1],
            "ano": data[2],
            "info_adicional": info
        }
        
        print("Venda cadastrada com sucesso!")
        return True
    except:
        print("Erro ao cadastrar a venda.")
        return False


# [3]
def excluir_venda(excluir_id: int) -> None:
    """
        Exclui uma determinada venda.
        * Recebe o id da venda a ser excluída.
        * Substitui os valores do banco de dados "db_vendas" por:
            db_vendas[excluir_id] = {
                "produtos": "",
                "valores": "",
                "quantidades": "",
                "cpf_comprador": "",
                "nome_vendedor": "",
                "dia": "",
                "mes": "",
                "ano": "",
                "info_adicional": "Venda cancelada"
            }
    """
    try:
        if len(db_vendas) != 0:
            for chave in db_vendas[excluir_id]:
                db_vendas[excluir_id][chave] = "" if chave != "info_adicional" else "Venda cancelada"
            print("Venda excluída com sucesso!")
        else:
            print("Não há vendas cadastradas para serem excluídas.")
    except:
        print("Erro ao excluir a venda.")
        

# [4]
def exibir_vendas() -> None:
    """
        Exibe as vendas cadastradas no banco de dados "db_vendas".
    """
    if len(db_vendas) == 0:
        print("Não há vendas cadastradas.")
        print()
    else:
        for id_venda in db_vendas:
            print(f"ID: {id_venda}")
            if db_vendas[id_venda]["info_adicional"] != "Venda cancelada":
                for elemento, valor in zip(db_vendas[id_venda].keys(), db_vendas[id_venda].values()):
                    print(f"{elemento}: {valor}")
            else:
                print("Venda cancelada.")
            print()
        
        
# [5]
# A função de editar vendas é tratada junto com a função de cadastro (2)


# [6]
def calcular_vendas_ano_mes(ano: int, mes: int) -> list:
    """
        Calcula o valor total e médio de vendas de uma determinada data (ano/mes).
        * Recebe o ano e o mês de interesse.
        * Retorna as seguintes listas:
            [soma_total, media], se for possível calcular.
            [0, 0], caso contrário.
    """
    try:
        # Obtendo uma lista com todas as vendas realizadas apenas na data de interesse.
        vendas_no_ano_mes = filtrar_vendas_ano_mes(ano, mes)

        if len(vendas_no_ano_mes) != 0:
            print(f"Quantidade de vendas realizadas na data {mes}/{ano}: {len(vendas_no_ano_mes)}")
            
            soma_individual = []
            for venda in vendas_no_ano_mes:
                # Multiplicando o valor de cada peça pela respectiva quantidade vendida.
                valor_x_qtd = [v*q for v,q in zip(venda["valores"], venda["quantidades"])]
                # Somando os valores.
                soma_individual.append(sum(valor_x_qtd))
            
            soma_total = sum(soma_individual)
            media = soma_total/len(vendas_no_ano_mes)
            print(f". Total vendido: R$ {soma_total},00")
            print(". Média por venda: R$ {:.2f}".format(media))
            return [soma_total, media]
        else:
            print(f"Nenhuma venda foi realizada na data {mes}/{ano}.")
            return [0,0]
    except:
        print(f"Erro ao calcular o total e média de vendas em {mes}/{ano}.")
        return [0,0]

    
# [7]  
def calcular_vendedor_ano_mes(ano: int, mes: int) -> list:
    """
        Calcula o vendedor da data mm/aaaa.
        * Recebe o ano e o mês de interesse.
        * Retorna uma lista com a seguinte estrutura:
            [{"nome_vendedor": faturamento_total}, [vendedor_do_mes]]
    """
    try:
        # Primeiro, filtramos apenas as vendas do mes/ano desejado.
        vendas_no_ano_mes = filtrar_vendas_ano_mes(ano, mes)
        
        # Depois, utilizamos a função reduce aplicada à função filtrar_vendedor().
        # A intenção é criarmos um dicionário com a seguinte estrutura:
        # {"vendedor1": [{venda1_realizada_por_ele}, {venda2_realizada_por_ele} ...],
        #  "vendedor2": [{venda1_realizada_por_ele}, {venda2_realizada_por_ele} ...],
        #  ...}
        vendas_filtro_vendedor = reduce(filtrar_vendedor, vendas_no_ano_mes, {})
        
        if len(vendas_filtro_vendedor) != 0:
            
            # Criando um dicionário do tipo {"nome_vendedor": total_vendido_por_ele}
            dict_total_vendedores = {}
            
            for pessoa in vendas_filtro_vendedor.keys():
                
                # Neste primeiro momento, farei:
                # {"nome_vendedor": [total_venda1, total_venda2, ...]}
                dict_total_vendedores[pessoa] = []
                
                for venda_específica in vendas_filtro_vendedor[pessoa]:
                    precos = venda_específica["valores"]
                    qtds = venda_específica["quantidades"]
                    preco_x_qtd = [p*q for p,q in zip(precos, qtds)]
                    dict_total_vendedores[pessoa].append(sum(preco_x_qtd))
                
                # Agora teremos:
                # {"nome_vendedor": total_vendido_por_ele}
                dict_total_vendedores[pessoa] = sum(dict_total_vendedores[pessoa])
                
            print(f"Vendedores que tiveram faturamento na data {mes}/{ano}:")
            for v,t in zip(dict_total_vendedores.keys(), dict_total_vendedores.values()):
                print(f". {v}: R$ {t},00")
            print()
            
            maior_faturamento = max(list(dict_total_vendedores.values()))
            print("Vendedor(es) do mês (aquele(s) com o maior faturamento):")
            
            # Iremos armazenar os vendedores do mês em uma lista porque
            # pode haver mais de um vendedor com o mesmo faturamento.
            vendedor_mes = [
                pessoa for pessoa in dict_total_vendedores.keys()
                if dict_total_vendedores[pessoa] == maior_faturamento
            ]
            
            for v in vendedor_mes:
                print(f". {v}: R$ {dict_total_vendedores[v]},00")
            
            return [dict_total_vendedores, vendedor_mes]
        else:
            print(f"Não houve nenhuma venda na data {mes}/{ano}.")
            return [{},[]]
    except:
        print(f"Erro ao calcular o vendedor na data {mes}/{ano}.")
        return [{},[]]
    

# [8]:
def calcular_produto_ano_mes(ano: int, mes: int) -> list:
    """
        Calcula o produto com maior faturamento na data mm/aaaa.
        * Recebe ano e data de interesse.
        * Retorna uma lista com a seguinte estrutura:
            [{"nome_produto": faturamento no período}, [produto_do_mes]]
    """
    
    try:
        # Primeiro, filtramos apenas as vendas do mes/ano desejado.
        vendas_ano_mes = filtrar_vendas_ano_mes(ano, mes)
        
        # Criando uma lista de produtos vendidos no período.
        produtos_vendidos = []
        for venda in vendas_ano_mes:
            for produto in venda["produtos"]:
                if produto not in produtos_vendidos: 
                    produtos_vendidos.append(produto)
        
        # Criando uma lista com os respectivos faturamentos dos produtos vendidos no período.
        valor_produtos_vendidos = []
        for produto in produtos_vendidos:
            valor_total_produto = 0
            for venda in vendas_ano_mes:
                if produto in venda["produtos"]:
                    index_produto = venda["produtos"].index(produto)
                    valor_produto = venda["valores"][index_produto]
                    qtd_produto = venda["quantidades"][index_produto]
                    valor_total_produto += valor_produto*qtd_produto
            valor_produtos_vendidos.append(valor_total_produto)
        
        if len(produtos_vendidos) != 0:
            # Criando um dicionário do tipo {"produto_vendido": "valor"}.
            dict_produto_valor = dict(zip(produtos_vendidos, valor_produtos_vendidos))
            
            print(f"Produtos vendidos na data {mes}/{ano}:")
            for p in dict_produto_valor:
                print(f". {p}: R$ {dict_produto_valor[p]},00")
            print()
            
            maior_faturamento = max(valor_produtos_vendidos)
            
            # Iremos armazenar os produtos do mês em uma lista porque
            # pode haver mais de um produto com o mesmo faturamento.
            produtos_maior_faturamento = []
            print("Produto(s) do mês (aquele(s) com maior faturamento):")
            for p in dict_produto_valor:
                if dict_produto_valor[p] == maior_faturamento:
                    print(f". {p}: R$ {dict_produto_valor[p]},00")
                    produtos_maior_faturamento.append(p)
            
            return [dict_produto_valor, produtos_maior_faturamento]
        else:
            print(f"Não foram vendidos produtos na data {mes}/{ano}:")
            return [{},[]]
    except:
        return [{},[]]

    
# [9]

# Vou precisar dessa função extra para salvar os arquivos nas opções 9 e 10 do menu.
def escreve_linha_csv(dict_venda: dict) -> dict:
    """
        Prepara o dicionário para escrita no arquivo csv.
        * Recebe um dicionário no estilo {0: {venda0}, 1: {venda1}, ...}
        * Retorna uma lista [{venda_x}] onde {venda_x} = {"id": x, "produtos": [produtos], ...}
    """
    try:
        lista_vendas = list(dict_venda.values())
        id_vendas = list(dict_venda.keys())

        # Como as chaves "produtos", "valores" e "quantidades" armazenam listas,
        # irei organiza-las em strings para facilitar a escrita no arquivo csv.
        for i in range(len(lista_vendas)):
            # Preciso também adicionar o campo "id" nos dicionários das vendas individuais.
            lista_vendas[i] = {"id": id_vendas[i]} | lista_vendas[i]

            for chave in lista_vendas[i]:
                if (chave=="produtos"):
                    lista_vendas[i][chave] = "; ".join(lista_vendas[i][chave])
                elif (chave=="valores") or (chave=="quantidades"):
                    temp_lista = [str(v) for v in lista_vendas[i][chave]]
                    lista_vendas[i][chave] = "; ".join(temp_lista)
            yield lista_vendas[i]
    except:
        print("Erro ao preparar o dicionário para escrita no arquivo csv.")


def salvar_db():
    """
        Salva o que está escrito no dicionário db_vendas no arquivo db_vendas.csv
    """
    try:
        if len(db_vendas) != 0:
            # Criando uma lista no estilo [{venda1}, {venda2}, ...] utilizando uma função geradora.
            lista_linhas = []
            for linha in escreve_linha_csv(db_vendas):
                lista_linhas.append(linha)

            with open("db_vendas.csv", 'w') as db:
                escritor = csv.DictWriter(db,
                                          fieldnames=lista_linhas[0].keys(),
                                          lineterminator='\n')
                escritor.writeheader()
                escritor.writerows(lista_linhas)

            print("Alterações salvas no arquivo db_vendas.csv com sucesso!")
        else:
            print("Não há vendas cadastradas.")
    except:
        print("Erro ao salvar as vendas no banco de dados.")
    
    
# [10]
def converter_dolar(cotacao: float) -> None:
    try:
        if len(db_vendas) != 0:
            # Criando um dicionário cópia de db_vendas mas com valores dos produtos em dólar:
            db_dolar = deepcopy(db_vendas)
            for id_venda in db_dolar:
                db_dolar[id_venda]["valores"] = [
                    round(valor/cotacao, 2) for valor in db_dolar[id_venda]["valores"]
                    if db_dolar[id_venda]["info_adicional"] != "Venda cancelada"
                ]

            # Criando uma lista no estilo [{venda1}, {venda2}, ...] utilizando uma função geradora
            lista_linhas = []
            for linha in escreve_linha_csv(db_dolar):
                lista_linhas.append(linha)
            
            nome_arquivo = "db_dolar_" + str(round(cotacao, 2)) + ".csv"
            
            with open(nome_arquivo, 'w') as db_dolar:
                escritor = csv.DictWriter(db_dolar,
                                          fieldnames=lista_linhas[0].keys(),
                                          lineterminator='\n')
                escritor.writeheader()
                escritor.writerows(lista_linhas)

            print(f"Valores convertidos para dólar. Arquivo {nome_arquivo} salvo com sucesso!")
        else:
            print(f"Nenhuma venda identificada. O arquivo {nome_arquivo} não foi salvo.")
    except:
        print("Erro ao converter e salvar arquivo.")

## Main

Resumo do sistema:

* Primeiramente, se há um arquivo `db_vendas.csv`, os dados são importados para o dicionário `db_vendas`. Caso contrário, `db_vendas = {}`.
* O usuário faz seu login no sistema.
* O usuário navega pelo menu fazendo cadastros ou alterações.
* O usuário pode salvar as alterações no arquivo `db_vendas.csv`.
* O usuário pode converter os valores do dicionário `db_vendas` para dólar e salvar em um arquivo `db_dolar.csv`.
    * __Se não for selecionada a opção `9` ou `10` do menu em algum momento, as alterações da sessão não serão salvas, permanecendo apenas os dados anteriores.__
* O usuário pode gerar relatórios (como exibir vendas, catálogo de produtos, vendedor do mês, etc), só que esses relatórios não são salvos em nenhum arquivo externo.

Exemplo de valor a ser registrado no arquivo `db_vendas.csv`:

| id | produtos | valores | quantidades | cpf_comprador | nome_vendedor | dia | mes | ano | info_adicional |
| -- | -- | -- | -- | -- | -- | -- | -- | -- | -- |
| 0 | Camisa; Calça | 90; 120 | 1; 2 | 12345678900 | Gabriel | 11 | 6 | 2023 | Pagamento a prazo |
| 1 |  |  |  |  |  |  |  |  | Venda cancelada |

In [6]:
# Importando o banco de dados e identificando o último ID de venda cadastrado.
print("Procurando o banco de dados \"db_vendas.csv\"...")
db_vendas, ultimo_id = importar_db()
print()

ultimo_id += 1 if ultimo_id != 0 else 0

print("##### Sistema de Gerenciamento de Vendas #####")

# Este loop verifica se o login será feito.
while True:
    realizar_login = input("\nRealizar login? (S/N):")
    
    if realizar_login.upper() == 'S':
        usuario = input("Usuário:")
        senha = input("Senha:")

        if login(usuario,senha):
            print(f"Bem-vindo(a), {usuario}!\n")
            exibe_menu()
            
            # Este loop refere-se às ações do usuário no menu.
            while True:
                resposta_usuario = resposta_menu()
                
                if resposta_usuario == 11:
                    print(f"Até breve, {usuario}!")
                    break
                    
                elif resposta_usuario == 0:
                    print()
                    exibe_menu()
            
                elif resposta_usuario == 1:
                    print("\n##### [1] Catálogo de produtos da loja:\n")
                    exibe_produtos()
                    print()
                
                # Cadastrar e editar uma venda está nesse mesmo bloco de código.
                elif (resposta_usuario == 2) or (resposta_usuario == 5):
                    pular_edicao = False
                    if resposta_usuario == 2:
                        print("\n##### [2] Cadastrar venda:\n")
                        print(f"ID a ser cadastrado: {ultimo_id}\n")
                    else:
                        print("\n##### [5] Editar venda cadastrada:\n")
                        if len(db_vendas) != 0:
                            status_id = False
                            while not(status_id):
                                id_editado = input("Informe o ID da venda a ser editada:")
                                status_id, msg_id, id_editado = verifica_id(id_editado)
                                if msg_id:
                                    print(msg_id)
                            print()
                        else:
                            print("Não há vendas cadastradas para serem editadas.\n")
                            pular_edicao = True
                    
                    if not pular_edicao:
                        # Input do nome dos produtos vendidos:
                        # Atenção! A primeira letra do nome de cada produto deve ser maiúscula!
                        status_produtos = False
                        while not(status_produtos):
                            produtos_vendidos = input("Nome do(s) produto(s) vendido(s) (separar por vírgula):")
                            # Verificando se o input está correto:
                            status_produtos, msg_produtos, lista_produtos = verifica_produtos(produtos_vendidos)
                            if msg_produtos:
                                print(msg_produtos)
                        print()

                        # Input das quantidades de produtos vendidos, respectivamente:
                        status_qtd = False
                        while not(status_qtd):
                            qtd = input("Quantidade do(s) produto(s) vendido(s), respectivamente (separar por vírgula):")
                            # Verificando se o input está correto:
                            status_qtd, msg_qtd, lista_qtd = verifica_qtd(qtd, len(lista_produtos))
                            if msg_qtd:
                                print(msg_qtd)
                        print()

                        cpf = input("CPF do comprador:")
                        print()

                        # Input da data:
                        status_data = False
                        while not(status_data):
                            data_venda = input("Data da venda (formato dd/mm/aaaa):")
                            # Verificando se o input está correto:
                            status_data, msg_data, lista_data = verifica_data(data_venda)
                            if msg_data:
                                print(msg_data)
                        print()

                        # Input das informações adicionais:
                        status_info = False
                        while not(status_info):
                            info = input("Informações adicionais:")
                            # Verificando se o input está correto:
                            status_info, msg_info = verifica_info_adicional(info)
                            if msg_info:
                                print(msg_info)
                        print()

                        if (resposta_usuario == 2):
                            status_cadastro = cadastrar_venda(
                                ultimo_id,
                                lista_produtos,
                                lista_qtd,
                                usuario, 
                                cpf,
                                lista_data, 
                                info
                            )

                            if status_cadastro:
                                ultimo_id += 1
                        else:
                            cadastrar_venda(id_editado,
                                            lista_produtos,
                                            lista_qtd,
                                            usuario, 
                                            cpf,
                                            lista_data, 
                                            info)
                        print()

                elif resposta_usuario == 3:
                    print("\n##### [3] Excluir venda:\n")
                    if len(db_vendas) != 0:
                        status_id = False
                        while not(status_id):
                            excluir_id = input("Informe o ID da venda a ser excluída:")
                            status_id, msg_id, excluir_id = verifica_id(excluir_id)
                            if msg_id:
                                print(msg_id)
                        print()
                        excluir_venda(excluir_id)
                        print()
                    else:
                        print("Não há vendas a serem excluídas.\n")
                    
                elif resposta_usuario == 4:
                    print("\n##### [4] Exibir vendas:\n")
                    exibir_vendas()
                
                elif resposta_usuario == 6:
                    print("\n##### [6] Calcular o valor total e médio das vendas de um mês:\n")
                    status_data = False
                    while not(status_data):
                        mes_ano = input("Informe o mês e o ano de interesse (formato mm/aaaa):")
                        # Verificando se o input está correto:
                        status_data, msg_data, lista_data = verifica_data(mes_ano, False)
                        if msg_data:
                            print(msg_data)
                    print()
                    soma_ano_mes, media_ano_mes = calcular_vendas_ano_mes(ano=lista_data[1],
                                                                          mes=lista_data[0])
                    print()
                
                elif resposta_usuario == 7:
                    print("\n##### [7] Vendedor do mês:\n")
                    status_data = False
                    while not(status_data):
                        mes_ano = input("Informe o mês e o ano de interesse (formato mm/aaaa):")
                        # Verificando se o input está correto:
                        status_data, msg_data, lista_data = verifica_data(mes_ano, False)
                        if msg_data:
                            print(msg_data)
                    print()
                    faturamento_periodo, vendedor_mes = calcular_vendedor_ano_mes(ano=lista_data[1],
                                                                                  mes=lista_data[0])
                    print()
                
                elif resposta_usuario == 8:
                    print("\n##### [8] Produto do mês:\n")
                    status_data = False
                    while not(status_data):
                        mes_ano = input("Informe o mês e o ano de interesse (formato mm/aaaa):")
                        # Verificando se o input está correto:
                        status_data, msg_data, lista_data = verifica_data(mes_ano, False)
                        if msg_data:
                            print(msg_data)
                    print()
                    produto_valor, produto_mes = calcular_produto_ano_mes(ano=lista_data[1],
                                                                          mes=lista_data[0])
                    print()
                    
                elif resposta_usuario == 9:
                    print("\n##### [9] Salvar as vendas no arquivo db_vendas.csv:\n")
                    salvar_db()
                    print()
                  
                elif resposta_usuario == 10:
                    print("\n##### [10] Converter os valores em dólares e salvar dados em um novo arquivo:\n")
                    status_cotacao = False
                    while not(status_cotacao):
                        # Deve ser informado um número que possa ser convertido para float.
                        cotacao = input("Quantos dólares está valendo R$ 1,00?")
                        # Verificando se o input está correto:
                        status_cotacao, msg_cotacao, cotacao = verifica_cotacao_dolar(cotacao)
                        if msg_cotacao:
                            print(msg_cotacao)
                    print()
                    salvar_db()
                    converter_dolar(cotacao)
                    print()
                    
                else:
                    print(resposta_usuario)
                    continue

        else:
            print("Usuário ou senha inválidos!")
            continue
    elif realizar_login.upper() == 'N':
        print("Sistema encerrado!")
        break
    else:
        print("Digite \'S\' ou \'N\'.")
        continue

Procurando o banco de dados "db_vendas.csv"...
Arquivo encontrado!

##### Sistema de Gerenciamento de Vendas #####



Realizar login? (S/N): s
Usuário: Gabriel
Senha: abc


Bem-vindo(a), Gabriel!

##### Menu #####

0. Exibe o menu novamente
1. Exibe o catálogo de produtos da loja
2. Cadastrar venda
3. Excluir venda
4. Exibir vendas cadastradas
5. Editar venda cadastrada
6. Calcular o valor total e o valor médio das vendas em um determinado mês
7. Vendedor do mês
8. Produto do mês
9. Salvar as vendas no arquivo db_vendas.csv
10. Converter os valores em dólares e salvar dados em um novo arquivo
	* Essa operação salva também as novas vendas cadastradas no arquivo db_vendas.csv.
11. Sair

Não se esqueça de salvar as alterações (opção 9) antes de sair!



O que deseja realizar? Número correspondente às opções do menu: 4



##### [4] Exibir vendas:

ID: 0
Venda cancelada.

ID: 1
Venda cancelada.

ID: 2
produtos: ['Camisa']
valores: [90]
quantidades: [1]
cpf_comprador: 123456789
nome_vendedor: Gabriel
dia: 11
mes: 12
ano: 2023
info_adicional: 



...


Alterações salvas no arquivo db_vendas.csv com sucesso!
Valores convertidos para dólar. Arquivo db_dolar_4.79.csv salvo com sucesso!



O que deseja realizar? Número correspondente às opções do menu: 11


Até breve, Maria!



Realizar login? (S/N): n


Sistema encerrado!
